In [ ]:
# --- Imports ---

import os
import re
import time
import json
import sqlite3
import voyageai
import numpy as np
import pandas as pd
from typing import Dict
import concurrent.futures
from openai import OpenAI
from tqdm.notebook import tqdm
from collections import defaultdict

client = OpenAI()

In [ ]:
# Load embeddings and texts from the database

def load_embeddings_from_db(db_path: str) -> pd.DataFrame:
    try:
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database not found: {db_path}")
            
        conn = sqlite3.connect(db_path)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        cursor.execute("SELECT id, content, source, embedding FROM chunks")
        rows = cursor.fetchall()
        
        if not rows:
            print(f"Warning: No chunks found in {db_path}")
            return pd.DataFrame()
            
        records = []
        for row in rows:
            try:
                # Convert BLOB back to numpy array
                embedding = np.frombuffer(row["embedding"], dtype=np.float32)
                records.append({
                    "id": row["id"], 
                    "document": row["content"], 
                    "source": row["source"], 
                    "embedding": embedding
                })
            except (ValueError, TypeError) as e:
                print(f"Warning: Skipping invalid row {row['id']}: {e}")
                continue
        
        df = pd.DataFrame(records)
        return df
        
    except Exception as e:
        print(f"Error loading embeddings from {db_path}: {e}")
        raise
    finally:
        if 'conn' in locals():
            conn.close()

df_sql = load_embeddings_from_db("../databases/voyage_3_large_nice_guidelines.db")

In [ ]:
df_sql.info()

In [ ]:
# Filter unwanted guideline sections
unwanted_patterns = [
    'Putting this guideline into practice',
    'Finding more information and committee details',
    'Update information',
    'Evaluation committee members and NICE project team',
    'Committee members',
    'Appendix',
    'About this quality standard',
    'Endorsing organisation',
    'Supporting organisations',
    'Diagnostics advisory committee members and NICE project team',
    'Appraisal committee members',
    'Sources of evidence',
    r'(The\s+)?[Cc]ommittee\'?s?\s+discussion' 
]

filtered_df = df_sql[df_sql['source'].str.startswith('CG') | df_sql['source'].str.startswith('NG')]
committee_pattern = r'(The\s+)?[Cc]ommittee\'?s?\s+discussion'
filtered_df = filtered_df[~filtered_df['id'].str.contains(committee_pattern, case=False, na=False, regex=True)]

other_sections = [p for p in unwanted_patterns if not p.startswith('(')]
for section in other_sections:
    filtered_df = filtered_df[~filtered_df['id'].str.contains(section, case=False, na=False)]

filtered_df = filtered_df[filtered_df['document'].str.len() >= 100]

print(f"Original rows: {len(df_sql)}")
print(f"Filtered rows: {len(filtered_df)}")
filtered_df.info()

In [ ]:
filtered_df2 = filtered_df.copy()

In [ ]:
# Generate queries for each chunk

def generate_query_batch(rows, model="gpt-4.1-nano"):
    results = {}
    for row in rows:
        doc_id = row['id']
        doc_content = row['document']
        prompt = f"""Document Excerpt: {doc_content}
                Generate a realistic search query for this NICE guideline content."""
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "system",
                        "content": """You are an expert assistant specialized in generating realistic search queries for NICE guidelines.

                        Your task: Generate a natural question that a healthcare professional or patient would realistically use to find the given information.

                        Requirements:
                        - Start with question words: 'What', 'How', 'When', 'Should', 'Which', 'Why', etc.
                        - Focus on the core clinical topic or specific recommendation
                        - Use natural medical terminology that real users would search for
                        - Keep it concise and directly related to the content
                        - Return only the question, no additional formatting or explanations

                        Examples:
                        - "What are the treatment options for managing hypertension in pregnant women?"
                        - "How should blood glucose be monitored in diabetes patients?"
                        - "When should antibiotics be prescribed for respiratory infections?"
                        - "Which medications are recommended for chronic pain management?"
                        """
                    },
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=60,
            )
            query = response.choices[0].message.content.strip()
            results[doc_id] = query
        except Exception as e:
            print(f"An error occurred for document {doc_id}: {e}")
            results[doc_id] = "NaN"
    return results

# Batch run of query generation

from tqdm.notebook import tqdm

def batch_generate_queries(df, batch_size=100, model="gpt-4.1-nano"):
    batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]
    all_results = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(generate_query_batch, batch.to_dict('records'), model) for batch in batches]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Generating queries"):
            all_results.update(future.result())
    return all_results

query_results = batch_generate_queries(filtered_df2, batch_size=20, model="gpt-4.1-nano")
filtered_df2["query"] = filtered_df2['id'].map(query_results)
filtered_df2.to_csv("databases/suggested_queries.csv", index=False)

In [ ]:
filtered_df2 = pd.read_csv("databases/suggested_queries.csv", index_col=False)
filtered_df2.head()

In [ ]:
# Create database with embeddings of generated questions using Voyage AI

def get_voyage_client():
    voyage_api_key = os.getenv("VOYAGE_API_KEY")
    return voyageai.Client(api_key=voyage_api_key)

def get_embeddings_batch(texts, client, model="voyage-3.5"):
    try:
        result = client.embed(
            texts=texts,
            model=model,
            input_type="query",
            output_dimension=2048,
            truncation=True
        )
        return result.embeddings
    except Exception as e:
        print(f"Error getting embeddings: {e}")
        raise

def init_query_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS queries (
            id TEXT PRIMARY KEY,
            query TEXT NOT NULL,
            query_embedding BLOB
        )
    ''')
    conn.commit()
    conn.close()

def store_query(conn, query_id, query, embedding):
    cursor = conn.cursor()
    embedding_blob = np.array(embedding, dtype=np.float32).tobytes() if embedding is not None else None
    cursor.execute("""
        INSERT OR REPLACE INTO queries (id, query, query_embedding)
        VALUES (?, ?, ?)
    """, (query_id, query, embedding_blob))
    conn.commit()

def create_query_embeddings_db(csv_file, model_name="voyage-3.5"):
    db_path = f"databases/{model_name.replace('-', '_')}_suggested_queries_2048.db"
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    init_query_database(db_path)

    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} queries from {csv_file}")

    voyage_client = get_voyage_client()
    conn = sqlite3.connect(db_path)

    batch_size = 100
    for i in tqdm(range(0, len(df), batch_size), desc="Processing query batches"):
        batch = df.iloc[i:i+batch_size]
        queries = batch["query"].fillna("").tolist()
        ids = batch["id"].tolist()
        try:
            embeddings = get_embeddings_batch(queries, voyage_client, model_name)
            for query_id, query, embedding in zip(ids, queries, embeddings):
                store_query(conn, query_id, query, embedding)
        except Exception as e:
            print(f"Error processing batch starting at index {i}: {e}")
            continue

    conn.close()
    print(f"Query database creation/update completed: {db_path}")

if __name__ == "__main__":
    csv_file = "databases/suggested_queries.csv"
    create_query_embeddings_db(csv_file)

In [ ]:
# Create database with embeddings of generated questions using OpenAI

filtered_df2 = pd.read_csv("databases/suggested_queries.csv")

def get_openai_client():
    """Get OpenAI client with API key from environment."""
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return OpenAI(api_key=openai_api_key)

def get_embeddings_batch(texts, client, model="text-embedding-3-large"):
    """Get embeddings for a batch of texts using OpenAI API."""
    try:
        response = client.embeddings.create(
            input=texts,
            model=model
        )
        embeddings = [item.embedding for item in response.data]
        return embeddings
    except Exception as e:
        print(f"Error getting embeddings: {e}")
        raise

def init_query_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS queries (
            id TEXT PRIMARY KEY,
            query TEXT NOT NULL,
            query_embedding BLOB
        )
    ''')
    conn.commit()
    conn.close()

def store_query(conn, query_id, query, embedding):
    cursor = conn.cursor()
    embedding_blob = np.array(embedding, dtype=np.float32).tobytes() if embedding is not None else None
    cursor.execute("""
        INSERT OR REPLACE INTO queries (id, query, query_embedding)
        VALUES (?, ?, ?)
    """, (query_id, query, embedding_blob))
    conn.commit()

def create_query_embeddings_db(csv_file, model_name="text-embedding-3-large"):
    db_path = f"databases/{model_name.replace('-', '_')}_suggested_queries.db"
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    init_query_database(db_path)

    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} queries from {csv_file}")

    openai_client = get_openai_client()
    conn = sqlite3.connect(db_path)

    batch_size = 100
    for i in tqdm(range(0, len(df), batch_size), desc="Processing query batches"):
        batch = df.iloc[i:i+batch_size]
        queries = batch["query"].fillna("").tolist()
        ids = batch["id"].tolist()
        try:
            embeddings = get_embeddings_batch(queries, openai_client, model_name)
            for query_id, query, embedding in zip(ids, queries, embeddings):
                store_query(conn, query_id, query, embedding)
        except Exception as e:
            print(f"Error processing batch starting at index {i}: {e}")
            continue

    conn.close()
    print(f"Query database creation/update completed: {db_path}")

if __name__ == "__main__":
    csv_file = "databases/suggested_queries.csv"
    create_query_embeddings_db(csv_file)

In [ ]:
# Test query database loading

def load_query_database(db_path):
    """Load queries and embeddings from the database"""
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row  
    cursor = conn.cursor()
    
    cursor.execute("SELECT id, query, query_embedding FROM queries")
    rows = cursor.fetchall()
    
    records = []
    for row in rows:
        embedding = None
        if row["query_embedding"]:
            embedding = np.frombuffer(row["query_embedding"], dtype=np.float32)
        
        records.append({
            "id": row["id"],
            "query": row["query"], 
            "embedding": embedding
        })
    
    conn.close()
    return pd.DataFrame(records)

# Voyage 3.5 model
df_queries = load_query_database("databases/voyage_3_5_suggested_queries.db")

# Voyage 3 Large model  
# df_queries = load_query_database("databases/voyage_3_large_suggested_queries_2048.db")

# OpenAI text-embedding-3-large
# df_queries = load_query_database("databases/text_embedding_3_large_suggested_queries.db")

print(f"Loaded {len(df_queries)} queries")
df_queries.head()